In [ ]:
import fastf1
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from matplotlib.cm import get_cmap
from matplotlib.colors import Normalize

fastf1.Cache.enable_cache('cache')

races = [
    {'year': 2023, 'gp': 'Bahrain',        'label': 'GP Bahrajnu'},
    {'year': 2023, 'gp': 'SaudiArabia',    'label': 'GP Arabii Saudyjskiej'},
    {'year': 2023, 'gp': 'Australia',      'label': 'GP Australii'},
    {'year': 2023, 'gp': 'Azerbaijan',     'label': 'GP Azerbejdżanu'},
    {'year': 2023, 'gp': 'Miami',          'label': 'GP Miami'},
    {'year': 2023, 'gp': 'EmiliaRomagna',  'label': 'GP Emilii-Romanii'},
    {'year': 2023, 'gp': 'Monaco',         'label': 'GP Monako'},
    {'year': 2023, 'gp': 'Spain',          'label': 'GP Hiszpanii'},
    {'year': 2023, 'gp': 'Canada',         'label': 'GP Kanady'},
    {'year': 2023, 'gp': 'Austria',        'label': 'GP Austrii'},
    {'year': 2023, 'gp': 'GreatBritain',   'label': 'GP Wielkiej Brytanii'},
    {'year': 2023, 'gp': 'Hungary',        'label': 'GP Węgier'},
    {'year': 2023, 'gp': 'Belgium',        'label': 'GP Belgii'},
    {'year': 2023, 'gp': 'Netherlands',    'label': 'GP Holandii'},
    {'year': 2023, 'gp': 'Italy',          'label': 'GP Włoch'},
    {'year': 2023, 'gp': 'Singapore',      'label': 'GP Singapuru'},
    {'year': 2023, 'gp': 'Japan',          'label': 'GP Japonii'},
    {'year': 2023, 'gp': 'Qatar',          'label': 'GP Kataru'},
    {'year': 2023, 'gp': 'USA',            'label': 'GP Stanów Zjednoczonych'},
    {'year': 2023, 'gp': 'Mexico',         'label': 'GP Meksyku'},
    {'year': 2023, 'gp': 'Brazil',         'label': 'GP Brazylii'},
    {'year': 2023, 'gp': 'LasVegas',       'label': 'GP Las Vegas'},
    {'year': 2023, 'gp': 'AbuDhabi',       'label': 'GP Abu Zabi'},
]

race_data = {}

for r in races:
    sesja = fastf1.get_session(r['year'], r['gp'], 'R')
    sesja.load()
    w = sesja.weather_data
    if 'TrackTemp' in w.columns and not w['TrackTemp'].empty:
        t = w['TrackTemp']
        race_data[r['label']] = {
            'gp': r['gp'],
            'manual': {
                'min': float(t.min()),
                'avg': float(t.mean()),
                'max': float(t.max())
            }
        }
    else:
        race_data[r['label']] = {
            'gp': r['gp'],
            'manual': {
                'min': None,
                'avg': None,
                'max': None
            }
        }

race_laps_data = {}

for r in races:
    try:
        sesja = fastf1.get_session(r['year'], r['gp'], 'R')
        sesja.load(laps=True, telemetry=False, weather=False, messages=False) 

        if sesja.laps is not None and not sesja.laps.empty:
            if sesja.event['EventName'] == 'Emilia Romagna Grand Prix' and r['year'] == 2023: 
                 num_laps = 'N/A' 
            elif hasattr(sesja, 'total_laps') and sesja.total_laps is not None: 
                num_laps = int(sesja.total_laps)
            elif sesja.laps is not None and not sesja.laps.empty: 
                num_laps = int(sesja.laps['LapNumber'].max())
            else: 
                num_laps = None

            race_laps_data[r['label']] = {
                'gp': r['gp'],
                'laps': num_laps
            }
        else:
            
            num_laps_fallback = None
            if sesja.event['EventName'] == 'Emilia Romagna Grand Prix' and r['year'] == 2023:
                num_laps_fallback = 'N/A'

            race_laps_data[r['label']] = {
                'gp': r['gp'],
                'laps': num_laps_fallback 
            }
    except Exception as e:
        print(f"Nie udało się przetworzyć danych dla {r['label']}: {e}")
        laps_value_on_error = None
        if 'Emilia Romagna Grand Prix' in r['gp'] and r['year'] == 2023: 
            laps_value_on_error = 'N/A'

        race_laps_data[r['label']] = {
            'gp': r['gp'],
            'laps': laps_value_on_error 
        }

label_maps = {
    'temperatura':     {0: 'zimny',      1: 'umiarkowany', 2: 'gorący'},
    'wilgotność':      {0: 'suchy',      1: 'normalna',    2: 'wilgotny'},
    'opady':           {0: 'suchy',      1: 'normalne',    2: 'deszczowy'},
    'ciśnienie':       {0: 'niskie',     1: 'normalne',    2: 'wysokie'},
    'wiatr':           {0: 'spokojny',   1: 'umiarkowany', 2: 'wietrzny'},
    'temperatura_toru':{0: 'niska',      1: 'umiarkowana', 2: 'wysoka'}
}

def get_fastf1_weather(r):
    sess = fastf1.get_session(r['year'], r['gp'], 'R')
    sess.load()
    w = sess.weather_data
    return {
        'AirTemp':     w['AirTemp'],
        'TrackTemp':   w['TrackTemp'] if 'TrackTemp' in w.columns else pd.Series(dtype=float),
        'Humidity':    w['Humidity'],
        'Pressure':    w['Pressure'],
        'WindSpeed':   w['WindSpeed'],
        'WindDirection': w['WindDirection'],
        'Rainfall':    w['Rainfall']
    }

def calculate_season_stats():
    records = []
    for r in races:
        try:
            w = get_fastf1_weather(r)
            records.append({
                'temperature':    w['AirTemp'].mean(),
                'track_temp':     w['TrackTemp'].mean() if not w['TrackTemp'].empty else np.nan,
                'humidity':       w['Humidity'].mean(),
                'pressure':       w['Pressure'].mean(),
                'wind':           w['WindSpeed'].mean(),
                'precipitation':  w['Rainfall'].sum()
            })
        except:
            continue
    df = pd.DataFrame(records)
    return df.mean().to_dict(), df.std().to_dict()

def classify_relative_to_season(w, mean, std, label):

    vals = {
        'temperature':    w['AirTemp'].mean(),
        'track_temp':     (w['TrackTemp'].mean() if not w['TrackTemp'].empty 
                           else race_data[label]['manual']['avg']),
        'humidity':       w['Humidity'].mean(),
        'pressure':       w['Pressure'].mean(),
        'wind':           w['WindSpeed'].mean(),
        'precipitation':  w['Rainfall'].sum()
    }
    codes = {}

    for cat in ['temperature','humidity','precipitation','pressure','wind','track_temp']:
        μ, σ = mean[cat], std[cat]
        x = vals[cat]
        codes[cat] = 2 if x > μ + σ else 0 if x < μ - σ else 1

    codes['wind_dir'] = int(w['WindDirection'].mode()[0])
    return codes

def plot_time_series(w, col, title, ylabel):
    series = w.get(col)
    if series is None or series.empty:
        return
    plt.figure(figsize=(8,3))
    plt.plot(series.index, series.values)
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel('Czas sesji')
    plt.grid(True)
    plt.show()

def plot_all(race):
    w = get_fastf1_weather(race)
    plot_time_series(w, 'AirTemp',        f"{race['label']} – Temperatura powietrza", "°C")
    plot_time_series(w, 'Humidity',       f"{race['label']} – Wilgotność",            "%")
    plot_time_series(w, 'Rainfall',       f"{race['label']} – Opady",                 "mm")
    plot_time_series(w, 'Pressure',       f"{race['label']} – Ciśnienie",             "hPa")
    plot_time_series(w, 'WindSpeed',      f"{race['label']} – Prędkość wiatru",       "km/h")

    temps = w.get('TrackTemp', pd.Series(dtype=float))
    if not temps.empty:
        plt.figure(figsize=(5,3))
        plt.bar(['Min','Średnia','Max'], [temps.min(), temps.mean(), temps.max()])
        plt.title(f"{race['label']} – Temperatura toru")
        plt.ylabel('°C')
        plt.show()

    dirs = w.get('WindDirection', pd.Series(dtype=float))
    speeds = w.get('WindSpeed', pd.Series(dtype=float))
    if not speeds.empty and not dirs.empty:
        cmap, normcol = get_cmap('hsv'), Normalize(0,360)
        plt.figure(figsize=(6,6))
        ax = plt.subplot(projection='polar')
        ax.scatter(np.radians(dirs), speeds, c=dirs, cmap=cmap, norm=normcol, s=50, alpha=0.7)
        ax.set_title(f"{race['label']} – Kierunek i siła wiatru")
        plt.show()

def main():
    mean_stats, std_stats = calculate_season_stats()

    print("Dostępne wyścigi 2023:")
    for r in races:
        print(" -", r['label'])
    wybor = input("Podaj etykietę wyścigu: ").strip()
    race = next((r for r in races if r['label']==wybor), None)
    if not race:
        print("Nie znaleziono wyścigu.")
        return

    w = get_fastf1_weather(race)
    codes = classify_relative_to_season(w, mean_stats, std_stats, race['label'])

    print(f"\n=== {race['label']} 2023 ===")
    mapping = {
        'temperature':'temperatura', 'humidity':'wilgotność',
        'precipitation':'opady', 'pressure':'ciśnienie',
        'wind':'wiatr', 'track_temp':'temperatura_toru'
    }
    for cat, code in codes.items():
        if cat == 'wind_dir':
            print(f"kierunek_wiatru: {code}°")
        else:
            key = mapping[cat]
            print(f"{key}: {code}: {label_maps[key][code]}")

    plot_all(race)

if __name__ == "__main__":
    main()



core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core           INFO 	Loading data for Saudi Arabian Grand Pri